In [7]:
import os
from dotenv import load_dotenv
import pandas as pd
import snowflake.connector

load_dotenv()

conn = snowflake.connector.connect(
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    # authenticator="externalbrowser",  # <- for SSO
    # private_key=...,                  # <- for key-pair auth
)

# Simple test query: swap table for what you actually need
sql = """
SELECT *
FROM SNACKLASH2.RAW.RSS_ARTICLES         -- your known table from past chats
ORDER BY PULLED_AT DESC
"""

df = pd.read_sql(sql, conn)
conn.close()

df.head(3)


/tmp/ipykernel_65350/122493978.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,URL,GUID,GUID_IS_PERMALINK,TITLE,SUMMARY,CONTENT_HTML,CONTENT_TEXT,AUTHOR_NAME,AUTHOR_EMAIL,AUTHOR_URI,...,IMAGE_URL,SOURCE_NAME,SOURCE_FEED_TITLE,SOURCE_FEED_URL,PUBLISHED_AT_RAW,UPDATED_AT_RAW,PULLED_AT,MATCHING_RULE_IDS,MATCHING_TERMS,RAW_XML
0,https://soundhealthandlastingwealth.com/chriss...,https://soundhealthandlastingwealth.com/?p=244...,False,Chrissy Teigen’s Emotional Weight Loss Journey...,<p>Chrissy Teigen took Ozempic to lose weight ...,<ul>\n<li><strong><em>Chrissy Teigen took Ozem...,<p>Chrissy Teigen took Ozempic to lose weight ...,shalw,None,None,...,None,Sound Health Blog,None,None,"Mon, 22 Sep 2025 14:05:34 +0000",None,2025-09-30 00:14:10.698680+00:00,"[\n 501,\n 402\n]",[],"<item xmlns:dc=""http://purl.org/dc/elements/1...."
1,https://www.bbc.com/news/videos/cvgrj408l2qo?a...,https://www.bbc.com/news/videos/cvgrj408l2qo#9,False,Are chia seeds and other fibre 'hacks' good fo...,Obesity and nutritional epidemiologist Dr Deir...,None,Obesity and nutritional epidemiologist Dr Deir...,None,None,None,...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,BBC - Health,None,None,"Wed, 24 Sep 2025 14:17:29 GMT",None,2025-09-30 00:14:10.698680+00:00,[\n 402\n],[],"<item xmlns:ns0=""http://search.yahoo.com/mrss/..."
2,https://www.bbc.com/news/articles/cre5xp83394o...,https://www.bbc.com/news/articles/cre5xp83394o#0,False,How weight-loss injections are turning obesity...,"With claims of a 'two-tier system', could it b...",None,"With claims of a 'two-tier system', could it b...",None,None,None,...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,BBC - Health,None,None,"Mon, 29 Sep 2025 07:02:26 GMT",None,2025-09-30 00:14:10.698680+00:00,[\n 402\n],[],"<item xmlns:ns0=""http://search.yahoo.com/mrss/..."


In [8]:
import os, json, base64
import streamlit as st
import pandas as pd
import boto3
import snowflake.connector
from botocore.exceptions import ClientError

SECRET_NAME = "snowflake-ec2-streamlit_connect"
REGION_NAME = "us-east-1"

# ---------- Secrets ----------
@st.cache_resource(show_spinner=False)
def get_secret_dict() -> dict:
    """Fetch and parse the JSON secret from AWS Secrets Manager."""
    try:
        sm = boto3.client("secretsmanager", region_name=REGION_NAME)
        resp = sm.get_secret_value(SecretId=SECRET_NAME)
    except ClientError as e:
        st.error(f"Failed to load secret '{SECRET_NAME}': {e}")
        st.stop()

    if "SecretString" in resp:
        data = json.loads(resp["SecretString"])
    else:
        data = json.loads(base64.b64decode(resp["SecretBinary"]))

    # quick sanity check
    required = [
        "SNOWFLAKE_ACCOUNT","SNOWFLAKE_USER","SNOWFLAKE_ROLE",
        "SNOWFLAKE_WAREHOUSE","SNOWFLAKE_DATABASE","SNOWFLAKE_SCHEMA",
        "PRIVATE_KEY_PEM_B64"
    ]
    missing = [k for k in required if not data.get(k)]
    if missing:
        st.error(f"Missing keys in secret: {missing}")
        st.stop()
    return data

# ---------- Snowflake connection (key-pair) ----------
@st.cache_resource(show_spinner=False)
def get_conn():
    sec = get_secret_dict()
    private_key = base64.b64decode(sec["PRIVATE_KEY_PEM_B64"])
    kwargs = dict(
        account   = sec["SNOWFLAKE_ACCOUNT"],      # e.g. zudcypc-noc91029
        user      = sec["SNOWFLAKE_USER"],         # APP_STREAMLIT
        role      = sec["SNOWFLAKE_ROLE"],
        warehouse = sec["SNOWFLAKE_WAREHOUSE"],
        database  = sec["SNOWFLAKE_DATABASE"],
        schema    = sec["SNOWFLAKE_SCHEMA"],
        private_key = private_key,
        client_session_keep_alive = True,
    )
    if sec.get("PRIVATE_KEY_PASSPHRASE"):
        kwargs["private_key_password"] = sec["PRIVATE_KEY_PASSPHRASE"]
    return snowflake.connector.connect(**kwargs)

# ---------- Query helper ----------
@st.cache_data(ttl=300, show_spinner=False)
def fetch_df(sql: str) -> pd.DataFrame:
    with get_conn() as conn:
        return pd.read_sql(sql, conn)

# ---------- UI ----------
st.title("Snowflake ↔ Streamlit (EC2)")
st.caption("Key-pair auth via AWS Secrets Manager")

# example query: adjust for your table(s)
n = st.slider("Rows", 5, 100, 25, step=5)
sql = f"""
SELECT *
FROM SNACKLASH2.RAW.RSS_ARTICLES
ORDER BY PULLED_AT DESC
LIMIT {int(n)}
"""
df = fetch_df(sql)
st.dataframe(df, use_container_width=True)


2025-10-01 17:15:23.118 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-01 17:15:23.119 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 17:15:23.140 
  command:

    streamlit run /home/joelmiddleton/snow_env/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-01 17:15:23.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 17:15:23.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 17:15:23.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 17:15:23.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 17:15:23.142 Thread 'Ma

UnboundLocalError: cannot access local variable 'resp' where it is not associated with a value